In [1]:
import pandas as pd
import sklearn
import itertools
import numpy as np
import seaborn as sb
import re
import nltk
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from sklearn.linear_model import PassiveAggressiveClassifier,LogisticRegression
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [2]:
train_df = pd.read_csv('train.csv')

In [3]:
train_df.head(15)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
6,6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,NaN,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
7,7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
8,8,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,87,12.0,Rheinhessen,NaN,NaN,Anna Lee C. Iijima,NaN,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel
9,9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam


In [4]:
train_df.shape

(129971, 14)

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


In [6]:
# def create_distribution(dataFile):
#     return sb.countplot(x='label', data=dataFile, palette='hls')

# #by calling below we can see that training, test and valid data seems to be failry evenly distributed between the classes
# create_distribution(train_df)
print("Quality",end='   ')
print("No of sample")
train_df['points'].value_counts()

Quality   No of sample


88     17207
87     16933
90     15410
86     12600
89     12226
91     11359
92      9613
85      9530
93      6489
84      6480
94      3758
83      3025
82      1836
95      1535
81       692
96       523
80       397
97       229
98        77
99        33
100       19
Name: points, dtype: int64

In [7]:
train_df=train_df.drop(['region_2'],axis=1)

In [8]:
def data_qualityCheck():
    print("{:{}}".format("\033[1mCOLUMN\033[0m",38),end='')
    print("{:{}}".format("\033[1mNULL VALUES COUNT\033[0m",18))
    for x in train_df.columns:
        print("{:{}}".format(x,34),end='')
        print(train_df[x].isnull().sum())

    
data_qualityCheck()

COLUMN                        NULL VALUES COUNT
Unnamed: 0                        0
country                           63
description                       0
designation                       37465
points                            0
price                             8996
province                          63
region_1                          21247
taster_name                       26244
taster_twitter_handle             31213
title                             0
variety                           1
winery                            0


In [9]:
print("{:{}}".format("\033[1mCOLUMN\033[0m",38),end='')
print("{:{}}".format("\033[1mUNIQUE VALUES COUNT\033[0m",18))
for x in train_df.columns:
        print("{:{}}".format(x,34),end='')
        print(len(train_df[x].unique()))

COLUMN                        UNIQUE VALUES COUNT
Unnamed: 0                        129971
country                           44
description                       119955
designation                       37980
points                            21
price                             391
province                          426
region_1                          1230
taster_name                       20
taster_twitter_handle             16
title                             118840
variety                           708
winery                            16757


In [10]:
train_df=train_df.drop(["region_1", "taster_twitter_handle","taster_name"], axis=1)

In [11]:
data_qualityCheck()

COLUMN                        NULL VALUES COUNT
Unnamed: 0                        0
country                           63
description                       0
designation                       37465
points                            0
price                             8996
province                          63
title                             0
variety                           1
winery                            0


In [12]:
def fill_data(data):
    data["country"] = data["country"].fillna("No Country")
    data["designation"] = data["designation"].fillna("No Designation")
    data["price"]=data["price"].fillna(0)
    data["province"]=data["province"].fillna("No Province")
    data["variety"]=data["variety"].fillna("No variety")
    return data

In [13]:
train_df=fill_data(train_df)

In [14]:
train_df.shape

(129971, 10)

In [15]:
train_df=train_df.drop(["Unnamed: 0"],axis=1)

In [16]:
train_df.head(20)

,country,description,designation,points,price,province,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,0.0,Sicily & Sardinia,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",No Designation,87,14.0,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
7,France,This dry and restrained wine offers spice in p...,No Designation,87,24.0,Alsace,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
8,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,87,12.0,Rheinhessen,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam


In [17]:
train_df.describe()

,points,price
count,129971.000000,129971.000000
mean,88.447138,32.915697
std,3.039730,40.582167
min,80.000000,0.000000
25%,86.000000,15.000000
50%,88.000000,25.000000
75%,91.000000,40.000000
max,100.000000,3300.000000


In [18]:
label_train = train_df['points']
label_train

0         87
1         87
2         87
3         87
4         87
          ..
129966    90
129967    90
129968    90
129969    90
129970    90
Name: points, Length: 129971, dtype: int64

In [19]:
label_train.head(10)

0    87
1    87
2    87
3    87
4    87
5    87
6    87
7    87
8    87
9    87
Name: points, dtype: int64

In [20]:
train_df = train_df.drop("points", axis = 1)

In [21]:
train_df.head(10)

,country,description,designation,price,province,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,0.0,Sicily & Sardinia,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",No Designation,14.0,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,13.0,Michigan,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,65.0,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,15.0,Northern Spain,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,16.0,Sicily & Sardinia,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
7,France,This dry and restrained wine offers spice in p...,No Designation,24.0,Alsace,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
8,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,12.0,Rheinhessen,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel
9,France,This has great depth of flavor with its fresh ...,Les Natures,27.0,Alsace,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam


In [22]:
data_qualityCheck()

COLUMN                        NULL VALUES COUNT
country                           0
description                       0
designation                       0
price                             0
province                          0
title                             0
variety                           0
winery                            0


In [23]:
train_df["text"]=train_df["country"]+" "+train_df["description"]+" "+train_df["designation"]+" "+train_df["province"]+" "+train_df["title"]+" "+train_df["variety"]+" "+train_df["winery"]

In [24]:
train_df=train_df.drop(["designation","country","province","description","title","variety","winery"],axis=1)

In [25]:
train_df.head(20)

,price,text
0,0.0,"Italy Aromas include tropical fruit, broom, br..."
1,15.0,"Portugal This is ripe and fruity, a wine that ..."
2,14.0,"US Tart and snappy, the flavors of lime flesh ..."
3,13.0,"US Pineapple rind, lemon pith and orange bloss..."
4,65.0,"US Much like the regular bottling from 2012, t..."
5,15.0,Spain Blackberry and raspberry aromas show a t...
6,16.0,"Italy Here's a bright, informal red that opens..."
7,24.0,France This dry and restrained wine offers spi...
8,12.0,Germany Savory dried thyme notes accent sunnie...
9,27.0,France This has great depth of flavor with its...


In [26]:
price=train_df["price"]
train_df=train_df.drop("price",axis=1)

In [27]:
custom_download_dir = "C:\\Users\\ysach/nltk"
nltk.data.path.append(custom_download_dir)

In [28]:
nltk.download('stopwords',download_dir=custom_download_dir)

[nltk_data] Downloading package stopwords to C:\Users\ysach/nltk...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
lemmatizer = WordNetLemmatizer()
stpwrds = list(stopwords.words('english'))

In [31]:
stpwrds

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [32]:
nltk.download('punkt',download_dir=custom_download_dir)
nltk.download('wordnet',download_dir=custom_download_dir)

[nltk_data] Downloading package punkt to C:\Users\ysach/nltk...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\ysach/nltk...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
nltk.download('omw-1.4',download_dir=custom_download_dir)

[nltk_data] Downloading package omw-1.4 to C:\Users\ysach/nltk...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [34]:
for x in range(len(train_df)) :
    corpus = []
    review = train_df['text'][x]
    review = re.sub(r'[^a-zA-Z\s]', '', review)
    review = review.lower()
    review = nltk.word_tokenize(review)
    for y in review :
        if y not in stpwrds :
            corpus.append(lemmatizer.lemmatize(y))
    review = ' '.join(corpus)
    train_df['text'][x] = review

In [35]:
train_df['text'][2188]

'u touch riesling accentuates fresh citrusy backbone cabernet sauvignon ro dry style sprightly lightfooted tone offer load concentrated cherry berry flavor finish brisk clean dry new york osprey dominion dry ro north fork long island ro osprey dominion'

In [36]:
label_train[2188]

89

In [37]:
X_train= train_df['text']

In [38]:
X_train

0         italy aroma include tropical fruit broom brims...
1         portugal ripe fruity wine smooth still structu...
2         u tart snappy flavor lime flesh rind dominate ...
3         u pineapple rind lemon pith orange blossom sta...
4         u much like regular bottling come across rathe...
                                ...                        
129966    germany note honeysuckle cantaloupe sweeten de...
129967    u citation given much decade bottle age prior ...
129968    france welldrained gravel soil give wine crisp...
129969    france dry style pinot gris crisp acidity also...
129970    france big rich offdry powered intense spicine...
Name: text, Length: 129971, dtype: object

In [39]:
X_train.shape

(129971,)

In [40]:
label_train.shape

(129971,)

In [45]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [46]:
tokenize = Tokenizer(oov_token="<OOV>")
tokenize.fit_on_texts(X_train)
word_idx = tokenize.word_index

text2seq = tokenize.texts_to_sequences(X_train)

# pad_seq = pad_sequences(text2seq, maxlen=150, padding="pre", truncating="pre")

pad_seq = pad_sequences(text2seq, padding="pre", truncating="pre")


print("The Padding Sequance Shape is  --> ", pad_seq.shape)

The Padding Sequance Shape is  -->  (129971, 89)


In [47]:
input_length = max(len(seq) for seq in text2seq)

vocabulary_size = len(word_idx) + 1

In [48]:
print("The maximum Sequance Length is  --> ", input_length)
print("The vocabulary size of dataset is  --> ", vocabulary_size)

The maximum Sequance Length is  -->  89
The vocabulary size of dataset is  -->  63971


In [49]:
df=pd.DataFrame(pad_seq)

In [50]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,88
0,0,0,0,0,0,0,0,0,0,0,...,824,429,449,11386,28260,824,1664,30,8,11386
1,0,0,0,0,0,0,0,0,0,0,...,1377,11387,11387,5,277,215,5,249,1377,11387
2,0,0,0,0,0,0,0,0,0,0,...,15,104,9316,11,225,373,9,11,225,9316
3,0,0,0,0,0,0,0,0,0,0,...,59,488,826,48,293,2608,4718,48,523,6353
4,0,0,0,0,0,0,0,0,0,0,...,7384,393,11,23,373,9,11,23,50,5969
5,0,0,0,0,0,0,0,0,0,0,...,36628,186,58,7385,6729,36628,14584,2103,14584,7385
6,0,0,0,0,0,0,0,0,0,0,...,571,74,11791,23585,3139,3853,3139,571,74,11791
7,0,0,0,0,0,0,0,0,0,0,...,52,290,440,15,172,6031,1064,172,446,6031
8,0,0,0,0,0,0,0,0,0,0,...,855,1135,7630,7922,855,446,1135,446,7630,7922
9,0,0,0,0,0,0,0,0,0,0,...,2141,77,724,11,225,172,11,225,3124,2141


In [51]:
df['89']=price

In [52]:
df.head(20)

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,0,0,0,0,0,0,0,0,0,0,...,429,449,11386,28260,824,1664,30,8,11386,0.0
1,0,0,0,0,0,0,0,0,0,0,...,11387,11387,5,277,215,5,249,1377,11387,15.0
2,0,0,0,0,0,0,0,0,0,0,...,104,9316,11,225,373,9,11,225,9316,14.0
3,0,0,0,0,0,0,0,0,0,0,...,488,826,48,293,2608,4718,48,523,6353,13.0
4,0,0,0,0,0,0,0,0,0,0,...,393,11,23,373,9,11,23,50,5969,65.0
5,0,0,0,0,0,0,0,0,0,0,...,186,58,7385,6729,36628,14584,2103,14584,7385,15.0
6,0,0,0,0,0,0,0,0,0,0,...,74,11791,23585,3139,3853,3139,571,74,11791,16.0
7,0,0,0,0,0,0,0,0,0,0,...,290,440,15,172,6031,1064,172,446,6031,24.0
8,0,0,0,0,0,0,0,0,0,0,...,1135,7630,7922,855,446,1135,446,7630,7922,12.0
9,0,0,0,0,0,0,0,0,0,0,...,77,724,11,225,172,11,225,3124,2141,27.0


In [53]:
zero_columns = df.columns[df.eq(0).all()]
print(zero_columns)

Index([], dtype='object')


In [54]:
df.shape

(129971, 90)

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(
    ngram_range=(1,1),
    max_features=25
)

In [58]:
df3=pd.get_dummies(label_train)

In [60]:
df3.head(10)

,80,81,82,83,84,85,86,87,88,89,...,91,92,93,94,95,96,97,98,99,100
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
!pip install tensorflow

In [43]:
import tensorflow as tf

In [62]:
df1=df['89']

In [63]:
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras import metrics as metrics1
from keras.layers import LeakyReLU
from keras.layers import Dense, Embedding, GlobalAveragePooling1D, LSTM, Bidirectional,InputLayer

In [64]:
x_train1, x_test, y_train1, y_test = train_test_split(pad_seq, df3, train_size=0.7)

In [65]:
classifier = Sequential()
classifier.add(Embedding(vocabulary_size, 200, input_length=89))
classifier.add(GlobalAveragePooling1D())
classifier.add(Dense(100, activation='relu'))
classifier.add(Dense(50, activation='relu'))
classifier.add(Dense(21, activation='sigmoid'))

# Compile the model
classifier.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [66]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 89, 200)           12794200  
                                                                 
 global_average_pooling1d (  (None, 200)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 100)               20100     
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 21)                1071      
                                                                 
Total params: 12820421 (48.91 MB)
Trainable params: 12820421 (48.91 MB)
Non-trainable params: 0 (0.00 Byte)
______________

In [67]:
classifier.fit(x_train1,y_train1,epochs=10,validation_data=(x_test, y_test))

Epoch 1/10


2844/2844 [==============================] - 700s 246ms/step - loss: 1.9759 - accuracy: 0.2405 - val_loss: 1.8103 - val_accuracy: 0.2790
Epoch 2/10
2844/2844 [==============================] - 680s 239ms/step - loss: 1.6719 - accuracy: 0.3298 - val_loss: 1.7734 - val_accuracy: 0.2931
Epoch 3/10
2844/2844 [==============================] - 716s 252ms/step - loss: 1.5275 - accuracy: 0.3928 - val_loss: 1.7976 - val_accuracy: 0.3054
Epoch 4/10
2844/2844 [==============================] - 744s 262ms/step - loss: 1.3634 - accuracy: 0.4765 - val_loss: 1.8590 - val_accuracy: 0.3115
Epoch 5/10
2844/2844 [==============================] - 702s 247ms/step - loss: 1.1777 - accuracy: 0.5636 - val_loss: 2.0084 - val_accuracy: 0.3067
Epoch 6/10
2844/2844 [==============================] - 651s 229ms/step - loss: 1.0069 - accuracy: 0.6370 - val_loss: 2.1700 - val_accuracy: 0.3127
Epoch 7/10
2844/2844 [==============================] - 636s 224ms/step - loss: 0.8697 - accuracy: 0.6905 - va

In [71]:
Y_pred = classifier.predict(x_test)
a=[]
for x in Y_pred:
    a.append(80 +np.argmax(x))

1219/1219 [==============================] - 1s 917us/step


In [72]:
a

[85,
 85,
 90,
 88,
 87,
 86,
 84,
 91,
 91,
 89,
 92,
 92,
 90,
 87,
 86,
 89,
 87,
 99,
 95,
 86,
 90,
 85,
 94,
 93,
 88,
 85,
 88,
 89,
 87,
 90,
 86,
 89,
 85,
 84,
 90,
 85,
 90,
 89,
 85,
 89,
 87,
 86,
 86,
 89,
 84,
 92,
 90,
 92,
 92,
 87,
 85,
 89,
 84,
 90,
 90,
 88,
 86,
 91,
 88,
 93,
 89,
 88,
 85,
 88,
 87,
 90,
 83,
 88,
 91,
 84,
 94,
 87,
 91,
 89,
 88,
 92,
 86,
 92,
 84,
 93,
 89,
 87,
 84,
 89,
 93,
 89,
 88,
 88,
 93,
 94,
 91,
 92,
 93,
 87,
 87,
 82,
 92,
 84,
 92,
 90,
 90,
 93,
 91,
 94,
 93,
 93,
 90,
 90,
 88,
 87,
 83,
 85,
 85,
 92,
 90,
 85,
 93,
 87,
 86,
 89,
 88,
 85,
 92,
 90,
 85,
 94,
 90,
 86,
 91,
 89,
 88,
 88,
 84,
 82,
 85,
 91,
 87,
 88,
 89,
 92,
 89,
 85,
 91,
 88,
 87,
 89,
 88,
 88,
 87,
 95,
 86,
 96,
 88,
 86,
 90,
 90,
 84,
 85,
 89,
 88,
 81,
 88,
 89,
 88,
 88,
 96,
 88,
 91,
 86,
 87,
 92,
 91,
 82,
 90,
 94,
 92,
 90,
 84,
 87,
 89,
 90,
 88,
 87,
 86,
 90,
 92,
 88,
 93,
 89,
 84,
 87,
 87,
 91,
 88,
 84,
 89,
 93,
 88,
 92,
 88,


In [74]:
from keras.layers import SimpleRNN,LSTM

In [75]:
model = Sequential()
model.add(Embedding(vocabulary_size, 100, input_length=input_length))
model.add(SimpleRNN(units=30, return_sequences=False))
model.add(Dense(units=21))

In [76]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [77]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 89, 100)           6397100   
                                                                 
 simple_rnn (SimpleRNN)      (None, 30)                3930      
                                                                 
 dense_3 (Dense)             (None, 21)                651       
                                                                 
Total params: 6401681 (24.42 MB)
Trainable params: 6401681 (24.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [81]:
model.fit(x_train1,y_train1,epochs=10,validation_data=(x_test, y_test))

Epoch 1/10
2844/2844 [==============================] - 352s 124ms/step - loss: 8.0596 - accuracy: 0.0731 - val_loss: 8.0594 - val_accuracy: 0.0759
Epoch 2/10
2844/2844 [==============================] - 413s 145ms/step - loss: 8.0596 - accuracy: 0.0731 - val_loss: 8.0594 - val_accuracy: 0.0759
Epoch 3/10
2844/2844 [==============================] - 342s 120ms/step - loss: 8.0596 - accuracy: 0.0731 - val_loss: 8.0594 - val_accuracy: 0.0759
Epoch 4/10
2844/2844 [==============================] - 344s 121ms/step - loss: 8.0596 - accuracy: 0.0731 - val_loss: 8.0594 - val_accuracy: 0.0759
Epoch 5/10
2844/2844 [==============================] - 342s 120ms/step - loss: 8.0596 - accuracy: 0.0731 - val_loss: 8.0594 - val_accuracy: 0.0759
Epoch 6/10
2844/2844 [==============================] - 353s 124ms/step - loss: 8.0596 - accuracy: 0.0731 - val_loss: 8.0594 - val_accuracy: 0.0759
Epoch 7/10
2844/2844 [==============================] - 378s 133ms/step - loss: 8.0596 - accuracy: 0.0731 - val_

In [82]:
Y_pred1 = classifier.predict(x_test)
a1=[]
for x in Y_pred1:
    a1.append(80 +np.argmax(x))

1219/1219 [==============================] - 1s 889us/step


In [83]:
a1

[85,
 85,
 90,
 88,
 87,
 86,
 84,
 91,
 91,
 89,
 92,
 92,
 90,
 87,
 86,
 89,
 87,
 99,
 95,
 86,
 90,
 85,
 94,
 93,
 88,
 85,
 88,
 89,
 87,
 90,
 86,
 89,
 85,
 84,
 90,
 85,
 90,
 89,
 85,
 89,
 87,
 86,
 86,
 89,
 84,
 92,
 90,
 92,
 92,
 87,
 85,
 89,
 84,
 90,
 90,
 88,
 86,
 91,
 88,
 93,
 89,
 88,
 85,
 88,
 87,
 90,
 83,
 88,
 91,
 84,
 94,
 87,
 91,
 89,
 88,
 92,
 86,
 92,
 84,
 93,
 89,
 87,
 84,
 89,
 93,
 89,
 88,
 88,
 93,
 94,
 91,
 92,
 93,
 87,
 87,
 82,
 92,
 84,
 92,
 90,
 90,
 93,
 91,
 94,
 93,
 93,
 90,
 90,
 88,
 87,
 83,
 85,
 85,
 92,
 90,
 85,
 93,
 87,
 86,
 89,
 88,
 85,
 92,
 90,
 85,
 94,
 90,
 86,
 91,
 89,
 88,
 88,
 84,
 82,
 85,
 91,
 87,
 88,
 89,
 92,
 89,
 85,
 91,
 88,
 87,
 89,
 88,
 88,
 87,
 95,
 86,
 96,
 88,
 86,
 90,
 90,
 84,
 85,
 89,
 88,
 81,
 88,
 89,
 88,
 88,
 96,
 88,
 91,
 86,
 87,
 92,
 91,
 82,
 90,
 94,
 92,
 90,
 84,
 87,
 89,
 90,
 88,
 87,
 86,
 90,
 92,
 88,
 93,
 89,
 84,
 87,
 87,
 91,
 88,
 84,
 89,
 93,
 88,
 92,
 88,


In [ ]:
#The first model performed better.

In [84]:
!pip install tensorflow-hub

In [100]:
def wine_quality_det(news):
    review = news
    review = re.sub(r'[^a-zA-Z\s]', '', review)
    review = review.lower()
    review = nltk.word_tokenize(review)
    for y in review :
        if y not in stpwrds :
            corpus.append(lemmatizer.lemmatize(y))     
    input_data = [' '.join(corpus)]
    vectorized_input_data_pre = tokenize.texts_to_sequences(input_data)
    vectorized_input_data=pad_sequences(vectorized_input_data_pre, padding="pre", truncating="pre")
    prediction = classifier.predict(vectorized_input_data)
    print(80 +np.argmax(prediction))

In [104]:
wine_quality_det("u touch riesling accentuates fresh citrusy backbone cabernet sauvignon ro dry style sprightly lightfooted tone offer load concentrated cherry berry flavor finish brisk clean dry new york osprey dominion dry ro north fork long island ro osprey dominion")

1/1 [==============================] - 0s 23ms/step
89
